In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pwd

/content


In [3]:
cd drive/MyDrive/PhoBert/

/content/drive/MyDrive/PhoBert


In [4]:
!pwd


/content/drive/MyDrive/PhoBert


In [5]:
#Load Intent Data
import numpy as np
import random
import json

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

with open('intents.json', 'r') as f:
    intents = json.load(f)

In [ ]:
intents

{'intents': [{'patterns': ['Xin chào',
    'Bạn khỏe không',
    'Có ai ở đó không?',
    'Xin chào',
    'Yolo',
    'Nàyyyy',
    'Aloha',
    'Có ai trả lời được không'],
   'responses': ['Xin chào!',
    'Chào :-)',
    'Xin chào, tôi có thể giúp gì cho bạn?',
    'Xin chào, tôi có thể giúp gì?',
    'Xin chào, cảm ơn đã ghé thăm',
    'Rất vui khi gặp lại'],
   'tag': 'greeting'},
  {'patterns': ['cya', 'Hẹn gặp lại', 'Tạm biệt', 'bye'],
   'responses': ['Hẹn gặp lại, cảm ơn đã ghé thăm',
    'Thật buồn khi bạn đi :(',
    'Chúc bạn ngày mới tốt lành',
    'Tạm biệt!',
    'Tạm biệt! Hãy Quay Lại Sớm Nhé.'],
   'tag': 'goodbye'},
  {'patterns': ['Cảm ơn',
    'Cảm ơn',
    'Thật hữu ích',
    'Cảm ơn rất nhiều!',
    'Thanks',
    'Thank you',
    'Thats helpful',
    "Thank's a lot!"],
   'responses': ['Vui vẻ giúp đỡ!', 'Hân hạnh'],
   'tag': 'thanks'},
  {'patterns': ['tên của bạn là gì',
    'tôi nên gọi bạn là gì',
    'bạn tên là gì?'],
   'responses': ['Bạn có thể gọi tôi l

In [6]:
all_words = []
tags = []
xy = []
# lặp qua intent để lưu tag and input training
for intent in intents['intents']:
    tag = intent['tag']
    # add to tag list
    tags.append(tag)
    for pattern in intent['patterns']:
        # tokenize each word in the sentence

        # add to our words list
        all_words.extend(pattern)
        # add to xy pair
        xy.append((pattern, tag))

In [7]:
x=[]
y=[]
for i in xy :
  x.append(i[0])
  y.append(i[1])
  

In [ ]:
x,y

(['Xin chào',
  'Bạn khỏe không',
  'Có ai ở đó không?',
  'Xin chào',
  'Yolo',
  'Nàyyyy',
  'Aloha',
  'Có ai trả lời được không',
  'cya',
  'Hẹn gặp lại',
  'Tạm biệt',
  'bye',
  'Cảm ơn',
  'Cảm ơn',
  'Thật hữu ích',
  'Cảm ơn rất nhiều!',
  'Thanks',
  'Thank you',
  'Thats helpful',
  "Thank's a lot!",
  'tên của bạn là gì',
  'tôi nên gọi bạn là gì',
  'bạn tên là gì?',
  'bạn có bất kỳ người bạn nào không ?',
  'bạn của bạn là ai ??',
  'Cậu có chấp nhận thanh toán qua thể tín dụng không?',
  'Bạn có chấp nhận thanh toán qua ATM không?',
  'Tôi có thể thanh toán bằng googlepay không?',
  'Bạn nhận thanh toán bằng tiền mặt?',
  'Bạn có chấp nhận thanh toán trực tuyến không.',
  'Tôi có thể đặt hàng thanh toán khi nhận hàng không',
  'Làm cách nào để thanh toán bằng tiền',
  'Bạn có những món đồ nào?',
  'Có những loại mặt hàng nào?',
  'Bạn bán gì?',
  'khi nào các bạn mở cửa',
  'mấy giờ bạn mở cửa',
  'giờ hoạt động',
  'Hãy kể cho tôi một câu chuyện cười!',
  'Hãy kể cho 

In [8]:
#visualize trên DataFrame
import pandas as pd
df = pd.DataFrame(list(zip(x, y)),
               columns =['text', 'label'])

In [9]:
df

,text,label
0,Xin chào,greeting
1,Bạn khỏe không,greeting
2,Có ai ở đó không?,greeting
3,Xin chào,greeting
4,Yolo,greeting
5,Nàyyyy,greeting
6,Aloha,greeting
7,Có ai trả lời được không,greeting
8,cya,goodbye
9,Hẹn gặp lại,goodbye


In [ ]:
#Clone BERT - transformers
!git clone https://github.com/huggingface/transformers


Cloning into 'transformers'...
remote: Enumerating objects: 109897, done.
remote: Total 109897 (delta 0), reused 0 (delta 0), pack-reused 109897
Receiving objects: 100% (109897/109897), 97.27 MiB | 7.08 MiB/s, done.
Resolving deltas: 100% (80155/80155), done.
Checking out files: 100% (2123/2123), done.


In [10]:
cd transformers/

/content/drive/MyDrive/PhoBert/transformers


In [17]:
pip install .

Processing /content/drive/MyDrive/PhoBert/transformers
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
ERROR: Operation cancelled by user


In [18]:
pip install transformers

     |████████████████████████████████| 4.0 MB 7.4 MB/s 
     |████████████████████████████████| 895 kB 55.0 MB/s 
     |████████████████████████████████| 77 kB 8.6 MB/s 
     |████████████████████████████████| 6.6 MB 55.8 MB/s 


In [12]:
pip install fastBPE

  Created wheel for fastBPE: filename=fastBPE-0.1.0-cp37-cp37m-linux_x86_64.whl size=483158 sha256=12c4c130f7aab7de546f1b68b9c5ea7c35d77708dbcd1f13c1bc8a6f87bd00b1
  Stored in directory: /root/.cache/pip/wheels/bd/d4/0e/0d317a65f77d3f8049fedd8a2ee0519164cf3e6bd77ef886f1
Successfully built fastBPE


In [13]:
pip install fairseq

     |████████████████████████████████| 1.7 MB 8.3 MB/s 
     |████████████████████████████████| 147 kB 71.8 MB/s 
     |████████████████████████████████| 90 kB 12.5 MB/s 
     |████████████████████████████████| 112 kB 75.9 MB/s 
     |████████████████████████████████| 74 kB 4.6 MB/s 
     |████████████████████████████████| 596 kB 57.8 MB/s 
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.8-py3-none-any.whl size=141230 sha256=f85b308d3aeadf646a37fa488476d0ec27dd6f6189bd756834a846018ec669ed
  Stored in directory: /root/.cache/pip/wheels/ca/33/b7/336836125fc9bb4ceaa4376d8abca10ca8bc84ddc824baea6c
Successfully built antlr4-python3-runtime
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
  Attempting uninstall: importlib-resources
    Found existing installation: importlib-resources 5.7.0
    Uninstalling importlib-resources-5.7.0:
      Successfully uninstalled

In [14]:
pip install vncorenlp

     |████████████████████████████████| 2.6 MB 10.4 MB/s 
  Created wheel for vncorenlp: filename=vncorenlp-1.0.3-py3-none-any.whl size=2645951 sha256=99c3e3decfc5e5a6ef07fd4d1f160d340bf40a6fcc7c681d714ccdc2803cbc9a
  Stored in directory: /root/.cache/pip/wheels/0c/d8/f2/d28d97379b4f6479bf51247c8dfd57fa00932fa7a74b6aab29
Successfully built vncorenlp


In [ ]:
!mkdir -p vncorenlp/models/wordsegmenter
!wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/VnCoreNLP-1.1.1.jar
!wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/wordsegmenter/vi-vocab
!wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/wordsegmenter/wordsegmenter.rdr
!mv VnCoreNLP-1.1.1.jar vncorenlp/ 
!mv vi-vocab vncorenlp/models/wordsegmenter/
!mv wordsegmenter.rdr vncorenlp/models/wordsegmenter/

--2022-04-20 07:30:25--  https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/VnCoreNLP-1.1.1.jar
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 27412575 (26M) [application/octet-stream]
Saving to: ‘VnCoreNLP-1.1.1.jar’

VnCoreNLP-1.1.1.jar 100%[===================>]  26.14M  42.6MB/s    in 0.6s    

2022-04-20 07:30:28 (42.6 MB/s) - ‘VnCoreNLP-1.1.1.jar’ saved [27412575/27412575]

--2022-04-20 07:30:28--  https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/wordsegmenter/vi-vocab
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting respo

In [15]:
!pwd


/content/drive/MyDrive/PhoBert/transformers


In [20]:
from vncorenlp import VnCoreNLP
rdrsegmenter = VnCoreNLP("/content/drive/MyDrive/PhoBert/transformers/vncorenlp/VnCoreNLP-1.1.1.jar", annotators="wseg", max_heap_size='-Xmx500m') 
# rdrsegmenter = VnCoreNLP("/content/drive/My Drive/BERT/SA/vncorenlp/VnCoreNLP-1.1.1.jar", annotators="wseg", max_heap_size='-Xmx500m') 

text = "Đại học Bách Khoa Hà Nội."

word_segmented_text = rdrsegmenter.tokenize(text) 
print(word_segmented_text)

[['Đại_học', 'Bách_Khoa', 'Hà_Nội', '.']]


In [21]:
text = "Đại học Bách Khoa Hà Nội."

word_segmented_text = rdrsegmenter.tokenize(text) 
print(word_segmented_text)

[['Đại_học', 'Bách_Khoa', 'Hà_Nội', '.']]


In [22]:
word_segmented_text = rdrsegmenter.tokenize(x[15]) 
print(word_segmented_text)

[['Cảm_ơn', 'rất', 'nhiều', '!']]


In [23]:
#prepare data
X_tokenize=[]
for sample in x:
  text = rdrsegmenter.tokenize(sample)
  text = ' '.join([' '.join(x) for x in text])
  X_tokenize.append(text)

In [24]:
splits = x[0].strip().split('\n')

In [25]:
splits

['Xin chào']

In [26]:
 text = ' '.join(splits[1:])[1:-1]

In [27]:
text

''

In [28]:
X_tokenize

['Xin chào',
 'Bạn khoẻ không',
 'Có ai ở đó không ?',
 'Xin chào',
 'Yolo',
 'Nàyyyy',
 'Aloha',
 'Có ai trả_lời được không',
 'cya',
 'Hẹn gặp lại',
 'Tạm_biệt',
 'bye',
 'Cảm_ơn',
 'Cảm_ơn',
 'Thật hữu_ích',
 'Cảm_ơn rất nhiều !',
 'Thanks',
 'Thank you',
 'Thats helpful',
 "Thank ' s a lot !",
 'tên của bạn là gì',
 'tôi nên gọi bạn là gì',
 'bạn tên là gì ?',
 'bạn có bất_kỳ người bạn nào không ?',
 'bạn của bạn là ai ? ?',
 'Cậu có chấp_nhận thanh_toán qua thể tín_dụng không ?',
 'Bạn có chấp_nhận thanh_toán qua ATM không ?',
 'Tôi có_thể thanh_toán bằng googlepay không ?',
 'Bạn nhận thanh_toán bằng tiền_mặt ?',
 'Bạn có chấp_nhận thanh_toán trực_tuyến không .',
 'Tôi có_thể đặt_hàng thanh_toán khi nhận hàng_không',
 'Làm cách nào để thanh_toán bằng tiền',
 'Bạn có những món đồ nào ?',
 'Có những loại mặt_hàng nào ?',
 'Bạn bán gì ?',
 'khi nào các bạn mở_cửa',
 'mấy giờ bạn mở_cửa',
 'giờ hoạt_động',
 'Hãy kể cho tôi một câu_chuyện cười !',
 'Hãy kể cho tôi nghe điều gì đó vui 

In [ ]:
!wget https://public.vinai.io/PhoBERT_base_fairseq.tar.gz
!tar -xzvf PhoBERT_base_fairseq.tar.gz

--2022-04-20 07:45:16--  https://public.vinai.io/PhoBERT_base_fairseq.tar.gz
Resolving public.vinai.io (public.vinai.io)... 54.192.19.64, 54.192.19.4, 54.192.19.25, ...
Connecting to public.vinai.io (public.vinai.io)|54.192.19.64|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1243308020 (1.2G) [application/x-tar]
Saving to: ‘PhoBERT_base_fairseq.tar.gz’

PhoBERT_base_fairse 100%[===================>]   1.16G  41.0MB/s    in 28s     

2022-04-20 07:45:45 (41.9 MB/s) - ‘PhoBERT_base_fairseq.tar.gz’ saved [1243308020/1243308020]

PhoBERT_base_fairseq/
PhoBERT_base_fairseq/bpe.codes
PhoBERT_base_fairseq/model.pt
PhoBERT_base_fairseq/dict.txt


In [ ]:
!wget https://public.vinai.io/PhoBERT_base_transformers.tar.gz
!tar -xzvf PhoBERT_base_transformers.tar.gz

--2022-04-20 07:47:19--  https://public.vinai.io/PhoBERT_base_transformers.tar.gz
Resolving public.vinai.io (public.vinai.io)... 54.192.19.88, 54.192.19.25, 54.192.19.64, ...
Connecting to public.vinai.io (public.vinai.io)|54.192.19.88|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 322405979 (307M) [application/x-tar]
Saving to: ‘PhoBERT_base_transformers.tar.gz’

PhoBERT_base_transf 100%[===================>] 307.47M  41.5MB/s    in 7.4s    

2022-04-20 07:47:26 (41.5 MB/s) - ‘PhoBERT_base_transformers.tar.gz’ saved [322405979/322405979]

PhoBERT_base_transformers/
PhoBERT_base_transformers/config.json
PhoBERT_base_transformers/bpe.codes
PhoBERT_base_transformers/model.bin
PhoBERT_base_transformers/dict.txt


In [29]:
from fairseq.data.encoders.fastbpe import fastBPE
from fairseq.data import Dictionary
import argparse

parser = argparse.ArgumentParser()
parser.add_argument('--bpe-codes', 
    default="/content/drive/MyDrive/PhoBert/transformers/PhoBERT_base_transformers/bpe.codes",
    required=False,
    type=str,
    help='path to fastBPE BPE'
)
args, unknown = parser.parse_known_args()
bpe = fastBPE(args)

# Load the dictionary
vocab = Dictionary()
vocab.add_from_file("/content/drive/MyDrive/PhoBert/transformers/PhoBERT_base_transformers/dict.txt")

In [30]:
#test encode
bpe.encode('Hôm_nay trời nóng quá nên tôi ở nhà viết Viblo!')

'Hôm_nay trời nóng quá nên tôi ở nhà viết Vi@@ blo@@ !'

In [31]:
bpe.encode(X_tokenize[40])

'Bạn có biết một câu_chuyện cười nào không ?'

In [32]:
vocab.encode_line('<s> ' + bpe.encode(X_tokenize[40]) + ' </s>')

tensor([  0, 970,  10,  55,  16, 876, 944, 142,  17, 114,   2,   2],
       dtype=torch.int32)

In [33]:
#ánh xạ X_tokenize thành mã 
from tensorflow.keras.preprocessing.sequence import pad_sequences
MAX_LEN = 125

train_ids = []
for sent in X_tokenize:
    subwords = '<s> ' + bpe.encode(sent) + ' </s>'
    encoded_sent = vocab.encode_line(subwords, append_eos=True, add_if_not_exist=False).long().tolist()
    train_ids.append(encoded_sent)

train_ids = pad_sequences(train_ids, maxlen=MAX_LEN, dtype="long", value=0, truncating="post", padding="post")

In [34]:
train_ids

array([[   0, 3280, 3683, ...,    0,    0,    0],
       [   0,  970, 3455, ...,    0,    0,    0],
       [   0,  503,  277, ...,    0,    0,    0],
       ...,
       [   0,  218,  505, ...,    0,    0,    0],
       [   0, 1584,    7, ...,    0,    0,    0],
       [   0, 1584,    7, ...,    0,    0,    0]])

In [35]:
#create Mask
train_masks = []
for sent in train_ids:
    mask = [int(token_id > 0) for token_id in sent]
    train_masks.append(mask)

In [ ]:
y_train =[]
for i in y:
  label = tags.index(i)
  y_train.append(label)
y_train

In [38]:
#tao luong dataloader
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
import torch

train_inputs = torch.tensor(train_ids)
train_masks = torch.tensor(train_masks)
y_trains = torch.tensor(y_train)

train_data = TensorDataset(train_inputs, train_masks, y_trains)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=32)



In [ ]:
!pwd

/content/drive/MyDrive/PhoBert/transformers


In [ ]:
from transformers import RobertaForSequenceClassification, RobertaConfig, AdamW,BertForSequenceClassification

config = RobertaConfig.from_pretrained(
    "/content/drive/MyDrive/PhoBert/transformers/PhoBERT_base_transformers/config.json", from_tf=False, num_labels = 14, output_hidden_states=False,
)
BERT_SA = RobertaForSequenceClassification.from_pretrained(
    "/content/drive/MyDrive/PhoBert/transformers/PhoBERT_base_transformers/model.bin",
    config=config
)
BERT_SA.cuda()

In [ ]:
#training
import random
from tqdm import tqdm_notebook
device = 'cuda'
epochs = 100

param_optimizer = list(BERT_SA.named_parameters())
no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay_rate': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay_rate': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=1e-5, correct_bias=False)

def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)


for epoch_i in range(0, epochs):
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    total_loss = 0
    BERT_SA.train()
    train_accuracy = 0
    nb_train_steps = 0
    train_f1 = 0
    
    for step, batch in tqdm_notebook(enumerate(train_dataloader)):
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        BERT_SA.zero_grad()
        outputs = BERT_SA(b_input_ids, 
            token_type_ids=None, 
            attention_mask=b_input_mask, 
            labels=b_labels)
        loss = outputs[0]
        total_loss += loss.item()
        
        logits = outputs[1].detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        tmp_train_accuracy = flat_accuracy(logits, label_ids)
        train_accuracy += tmp_train_accuracy
        nb_train_steps += 1
        
        loss.backward()
        torch.nn.utils.clip_grad_norm_(BERT_SA.parameters(), 1.0)
        optimizer.step()   
    avg_train_loss = total_loss / len(train_dataloader)
    print(" Accuracy: {0:.4f}".format(train_accuracy/nb_train_steps))
    print(" Average training loss: {0:.4f}".format(avg_train_loss))
print("Training complete!")

======== Epoch 1 / 100 ========
Training...


/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:32: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


0it [00:00, ?it/s]

 Accuracy: 0.3013
 Average training loss: 2.2842
======== Epoch 2 / 100 ========
Training...


0it [00:00, ?it/s]

...

 Accuracy: 1.0000
 Average training loss: 0.0261
======== Epoch 99 / 100 ========
Training...


0it [00:00, ?it/s]

 Accuracy: 1.0000
 Average training loss: 0.0260
======== Epoch 100 / 100 ========
Training...


0it [00:00, ?it/s]

 Accuracy: 1.0000
 Average training loss: 0.0251
Training complete!


In [ ]:
data = {
"model_state": BERT_SA.state_dict(),
}

In [ ]:
data

{'model_state': OrderedDict([('roberta.embeddings.position_ids',
               tensor([[  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
                         14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,  27,
                         28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,  40,  41,
                         42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,  53,  54,  55,
                         56,  57,  58,  59,  60,  61,  62,  63,  64,  65,  66,  67,  68,  69,
                         70,  71,  72,  73,  74,  75,  76,  77,  78,  79,  80,  81,  82,  83,
                         84,  85,  86,  87,  88,  89,  90,  91,  92,  93,  94,  95,  96,  97,
                         98,  99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111,
                        112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125,
                        126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 

In [ ]:
FILE = "data.pth"
torch.save(data, FILE)

print(f'training complete. file saved to {FILE}')

training complete. file saved to data.pth


In [114]:
#@title Thiết lập luồng input của bot step by step

In [94]:
#tokenize input - Do tiếng viết đa phần là từ ghép nên tokenize sẽ giúp chúng ta có ghép 2 từ thành 1 từ ghép có nghĩa.
# Example : 'bao', 'nhiêu' --> 'bao_nhiêu'
X = rdrsegmenter.tokenize("Giá của Tshirt là bao nhiêu")


[['Giá', 'của', 'Tshirt', 'là', 'bao_nhiêu']]

In [95]:
#Ghép lại thành string
X = ' '.join([' '.join(x) for x in X])
X

'Giá của Tshirt là bao_nhiêu'

In [96]:
#Khai báo subword đầu câu và cuối câu. Sau đó encode từ Vocab có sẵn( nhớ khai báo vocab)
subwords = '<s> ' + bpe.encode(X) + ' </s>'
encoded_sent = vocab.encode_line(subwords, append_eos=True, add_if_not_exist=False).long().tolist()
encoded_sent = [encoded_sent]
encoded_sent

[[0, 1584, 7, 982, 4014, 48503, 8, 1823, 2, 2]]

In [97]:
# đặt pad_sequences dựa trên câu dài nhất. Tuy nhiên maxlen ta nên chọn đủ lớn
X = pad_sequences(encoded_sent, maxlen = 125, dtype="long", value=0, truncating="post", padding="post")
X

array([[    0,  1584,     7,   982,  4014, 48503,     8,  1823,     2,
            2,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0]])

In [98]:
#tạo attention mask
mask = [int(token_id > 0) for token_id in X[0]]
mask = np.array([mask])
mask

array([[0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [ ]:
#Tạo device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

with open('/content/drive/MyDrive/PhoBert/intents.json', 'r') as json_data:
    intents = json.load(json_data)

FILE = "data.pth"
data = torch.load(FILE)

model_state = data["model_state"]
model = RobertaForSequenceClassification.from_pretrained(
    "/content/drive/MyDrive/PhoBert/transformers/PhoBERT_base_transformers/model.bin",
    config=config
)

model.load_state_dict(model_state)
model.eval()


In [ ]:
#đẩy model to GPU
model.to(device)

In [100]:
device = 'cuda'
b_input_ids = torch.from_numpy(X).to(device)
b_input_mask = torch.from_numpy(mask).to(device)


In [101]:
with torch.no_grad():
  outputs = model(b_input_ids, 
  token_type_ids=None, 
  attention_mask=b_input_mask)
  logits = outputs[0]
  logits = logits.detach().cpu().numpy()


In [107]:
outputs
probs = torch.softmax(outputs[0], dim=1)
probs

tensor([[4.7161e-04, 6.2055e-04, 4.1777e-04, 5.0258e-04, 3.0603e-04, 6.3842e-04,
         8.3487e-04, 7.5210e-04, 1.2120e-03, 1.2179e-03, 6.4241e-04, 5.7441e-04,
         1.3835e-03, 9.9043e-01]], device='cuda:0')

In [102]:

preds = np.argmax(logits, axis = 1)

In [112]:
preds.item()
tag = tags[preds.item()]
tag

'Tshirt'

In [113]:
#predict and run bot
bot_name = "Pizza Shop Bot"
print("Let's chat! (type 'quit' to exit)")
while True:
    # sentence = "do you use credit cards?"
    sentence = input("You: ")
    if sentence == "quit":
        break
    X = rdrsegmenter.tokenize(sentence)
    X = ' '.join([' '.join(x) for x in X])

    subwords = '<s> ' + bpe.encode(X) + ' </s>'
    encoded_sent = vocab.encode_line(subwords, append_eos=True, add_if_not_exist=False).long().tolist()
    encoded_sent = [encoded_sent]
    encoded_sent

    X = pad_sequences(encoded_sent, maxlen = 125, dtype="long", value=0, truncating="post", padding="post")

    mask = [int(token_id > 0) for token_id in X[0]]
    mask = np.array([mask])

    device = 'cuda'
    b_input_ids = torch.from_numpy(X).to(device)
    b_input_mask = torch.from_numpy(mask).to(device)

    with torch.no_grad():
      outputs = model(b_input_ids, 
      token_type_ids=None, 
      attention_mask=b_input_mask)
      logits = outputs[0]
      logits = logits.detach().cpu().numpy()

    preds = np.argmax(logits, axis = 1)
    #preds = array()
    tag = tags[preds.item()]

    probs = torch.softmax(outputs[0], dim=1)
    prob = probs[0][preds.item()]
    if prob.item() > 0.75:
        for intent in intents['intents']:
            if tag == intent["tag"]:
                print(f"{bot_name}: {random.choice(intent['responses'])}")
    else:
        print(f"{bot_name}: I do not understand...")

Let's chat! (type 'quit' to exit)
You: xin chao
Pizza Shop Bot: Chào :-)
You: bạn bán gì
Pizza Shop Bot: I do not understand...
You: tên bạn là gì
Pizza Shop Bot: Bạn có thể gọi tôi là Vịt hoặc Vịt Donald.
You: kể cho tôi chuyện cười
Pizza Shop Bot: You have two parts of the brain, “left” and “right”. On the left side, there’s nothing right and on the right side, there’s nothing left.
You: Áo bạn bán bao nhiêu
Pizza Shop Bot: Mức giá là Nhỏ-45.000vnd, Vừa-55.000vnd và Lớn-65.000vnd.
You: Quần bạn bán bao nhiêu
Pizza Shop Bot: 35.000 đồng cho size nhỏ là 45.000 đồng cho size Trung bình và 55.000 đồng cho size lớn.
You: bye
Pizza Shop Bot: Hẹn gặp lại, cảm ơn đã ghé thăm
You: tạm biệt
Pizza Shop Bot: Chúc bạn ngày mới tốt lành
You: bạn của bạn là ai
Pizza Shop Bot: Chuột mặc quần và chó tai dài
You: micky đâu rồi
Pizza Shop Bot: Anh ấy đang đi chơi với Mini
You: tôi có thể thanh toán bằng gì
Pizza Shop Bot: Nếu bạn không sử dụng phương thức thanh toán trực tuyến, bạn có thể thanh toán ch